# Задание 6. Классификация новостей

### Данные
Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

## Задание 6.1 

Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
## Задание 6.2

Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия
    

## Задание 6.4* 

Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

## Задание 6.1

Считываем данные

In [332]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("news_train.txt",sep="\t", header=None)

In [333]:
train_df

,0,1,2
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...
...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,Опубликован рейтинг лучших европейских пляжей ...
14996,media,В «Снобе» объяснили причину смены формата,Генеральный директор «Сноб медиа» Марина Гевор...
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,"Минфин разработал законопроект, устанавливающи..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,Актер и режиссер Мэл Гибсон выплатит своей быв...


Предобработка текста и его токенизация с использованием Byte-Pair Encoding

In [334]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
import re
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
train_df[2] = train_df[2].map(preprocess_string)
tokenizer.train_from_iterator(train_df[2], trainer=trainer)

def preprocess_string(string):
    prep_str = string.lower()
    prep_str = re.sub(r'[\d\.[\]«»,"\'%-?:—!;\(\)*]', ' ', prep_str)
    prep_str = re.sub(r'\s{2,}', ' ', prep_str) 
    return prep_str

def get_encoded1(text):
        return ' '.join(tokenizer.encode(text).tokens)
    
def get_encoded2(text):
    
    return tokenizer.encode(text).tokens

def preprocess_data(df):
    df[2] = df[2].map(preprocess_string)
    df = df.rename(columns={0:'label',1:'title',2:'text'})
    df['merged_text_tokens'] = df['text'].map(get_encoded1)
    df['text_tokens'] = df['text'].map(get_encoded2)
    return df

In [335]:
train_df = preprocess_data(train_df)

In [336]:
train_df

,label,title,text,merged_text_tokens,text_tokens
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,нападающий вашингтон кэпиталз александр овечки...,нападающий вашингтон кэпита лз александр овечк...,"[нападающий, вашингтон, кэпита, лз, александр,..."
1,culture,Рекордно дорогую статую майя признали подделкой,власти мексики объявили подделкой статую майя ...,власти мексики объявили поддел кой стату ю май...,"[власти, мексики, объявили, поддел, кой, стату..."
2,science,Samsung представила флагман в защищенном корпусе,южнокорейская samsung анонсировала защищенную ...,южнокорейская samsung анонсировала защищен ную...,"[южнокорейская, samsung, анонсировала, защищен..."
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,контрольно дисциплинарный комитет кдк рфс снял...,контрольно дисциплинарный комитет кдк рфс снял...,"[контрольно, дисциплинарный, комитет, кдк, рфс..."
4,media,Hopes & Fears объединится с The Village,интернет издание hopes fears объявило о своем ...,интернет издание hop es fe ars объявило о свое...,"[интернет, издание, hop, es, fe, ars, объявило..."
...,...,...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,опубликован рейтинг лучших европейских пляжей ...,опубликован рейтинг лучших европейских пля жей...,"[опубликован, рейтинг, лучших, европейских, пл..."
14996,media,В «Снобе» объяснили причину смены формата,генеральный директор сноб медиа марина геворкя...,генеральный директор сноб медиа марина ге во р...,"[генеральный, директор, сноб, медиа, марина, г..."
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,минфин разработал законопроект устанавливающий...,минфин разработал законопроект устанавли вающи...,"[минфин, разработал, законопроект, устанавли, ..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,актер и режиссер мэл гибсон выплатит своей быв...,актер и режиссер мэ л гиб сон выплатит своей б...,"[актер, и, режиссер, мэ, л, гиб, сон, выплатит..."


Пример токенизации

In [337]:
tokenizer.encode(train_df['text'][0]).tokens

['нападающий',
 'вашингтон',
 'кэпита',
 'лз',
 'александр',
 'овечкин',
 'передал',
 'детской',
 'хоккейной',
 'школе',
 'автомобиль',
 'полученный',
 'им',
 'после',
 'окончания',
 'матча',
 'всех',
 'звезд',
 'национальной',
 'хоккейной',
 'лиги',
 'нхл',
 'об',
 'этом',
 'сообщается',
 'на',
 'официальном',
 'сайте',
 'лиги',
 'автомобиль',
 'hon',
 'da',
 'ac',
 'cor',
 'd',
 'был',
 'пода',
 'рен',
 'хоккеи',
 'сту',
 'по',
 'решению',
 'спонсоров',
 'мероприятия',
 'игрок',
 'нхл',
 'пожертво',
 'вал',
 'машину',
 'спортивной',
 'школе',
 'no',
 'va',
 'c',
 'ool',
 'cats',
 'spe',
 'cial',
 'ho',
 'ck',
 'ey',
 'inc',
 'которая',
 'расположена',
 'в',
 'штате',
 'вирджи',
 'ния',
 'овечкин',
 'общается',
 'с',
 'летней',
 'дево',
 'чкой',
 'ан',
 'ной',
 'ш',
 'об',
 'с',
 'син',
 'дро',
 'мом',
 'да',
 'уна',
 'которая',
 'занимается',
 'в',
 'этой',
 'школе',
 'и',
 'является',
 'поклон',
 'ницей',
 'спортсмена',
 'в',
 'сентябре',
 'форвард',
 'по',
 'обе',
 'дал',
 'вместе'

## Задание 6.2

Обучение Word2Vec на полученных токенах

In [338]:
from gensim.models import Word2Vec

In [379]:
model = Word2Vec(sentences=train_df['text_tokens'], vector_size=100, window=5, min_count=1, workers=4)

In [380]:
vector = model.wv['интернет']
vector

array([-0.2713456 ,  0.75224817,  0.59428364, -0.6661041 ,  0.08521448,
       -1.1017848 , -0.12180103,  0.2086696 , -0.36816072, -1.0579935 ,
        0.14635268, -3.5296223 , -0.90736055, -0.03458572, -0.5147749 ,
       -1.3134733 ,  0.74391466, -0.17755744, -1.7102655 ,  0.6313617 ,
       -0.42276067,  0.09915036,  0.6468228 , -0.8001307 , -1.2569528 ,
       -0.04318319, -1.482901  , -0.20788544, -1.31753   , -0.62850875,
       -0.05660382, -0.31488466,  1.109137  , -0.05896315, -1.2651093 ,
        2.0284936 , -0.15981647,  0.08660134, -0.5327433 ,  2.0972307 ,
        0.3127952 , -0.22878267,  1.1684858 ,  1.2634224 , -0.30763727,
        1.2356262 ,  1.2154336 , -1.6669316 , -1.2354722 , -1.0382768 ,
       -0.9579463 ,  0.52561754,  0.12588295,  0.71330774, -0.2549415 ,
       -1.0028055 ,  2.3972318 , -1.2096295 ,  0.23734261,  0.30219495,
        0.04685164, -2.617919  , -0.04401742, -0.04873738,  0.47371677,
       -0.48354405, -0.5552493 ,  1.6642933 , -1.494386  ,  0.26

Поиск близких по смыслу токенов

In [381]:
model.wv.most_similar(tokenizer.encode('деньги').tokens, topn=10)

[('средства', 0.8091498017311096),
 ('платить', 0.7652372121810913),
 ('банки', 0.7526605129241943),
 ('налоги', 0.7471795082092285),
 ('свои', 0.7327820658683777),
 ('кредиты', 0.7322941422462463),
 ('вырученные', 0.7310887575149536),
 ('штрафы', 0.7225716710090637),
 ('денег', 0.7202186584472656),
 ('финансовые', 0.7194145321846008)]

In [382]:
model.wv.most_similar(tokenizer.encode('интернет').tokens, topn=10)

[('соцсети', 0.7878230810165405),
 ('google', 0.7545526027679443),
 ('сервис', 0.7533242702484131),
 ('сети', 0.7499057054519653),
 ('сервиса', 0.7421565651893616),
 ('яндекс', 0.7381110787391663),
 ('онлайн', 0.7365279197692871),
 ('микроблогов', 0.7287245392799377),
 ('ebay', 0.7193353772163391),
 ('yahoo', 0.7118775248527527)]

## Задание 6.3

Подготовим данные

Подгружаем тестовые данные и предобрабатываем их

In [344]:
test_df = pd.read_csv("news_test.txt",sep="\t", header=None)
test_df = preprocess_data(test_df)

In [345]:
test_df

,label,title,text,merged_text_tokens,text_tokens
0,culture,Жительница Ямала победила в первом песенном ко...,жительница ямало ненецкого автономного округа ...,жительница я мало нен ец кого автоном ного окр...,"[жительница, я, мало, нен, ец, кого, автоном, ..."
1,media,Почти половина Twitter-пользователей никогда н...,около процентов из всех зарегистрированных в t...,около процентов из всех зарегистрированных в t...,"[около, процентов, из, всех, зарегистрированны..."
2,media,"Билайн начал рекламу роуминга под песенку ""Тро...",в новой рекламной кампании мобильного оператор...,в новой рекламной кампании мобильного оператор...,"[в, новой, рекламной, кампании, мобильного, оп..."
3,business,"Saipem потеряла 1,2 миллиарда евро из-за отмен...",дочерняя структура итальянского нефтегазового ...,дочерняя структура итальянского нефтегазо вого...,"[дочерняя, структура, итальянского, нефтегазо,..."
4,culture,Вин Дизель назвал «Форсаж 7» достойным «Оскара»,актер вин дизель заявил что боевик форсаж в ко...,актер вин дизель заявил что боевик форсаж в ко...,"[актер, вин, дизель, заявил, что, боевик, форс..."
...,...,...,...,...,...
2995,science,"Причиной ""влажного"" климата Титана оказались м...",ученые прояснили причины влажного климата тита...,ученые проясни ли причины вла жного климата ти...,"[ученые, проясни, ли, причины, вла, жного, кли..."
2996,life,Британка нашла геккона в пакете с брокколи,жительница великобритании нашла геккона в паке...,жительница великобритании нашла ге к кона в па...,"[жительница, великобритании, нашла, ге, к, кон..."
2997,business,Владелец «Мечела» предложил закрыть в России в...,совладелец горно металлургического холдинга ме...,совладелец горно металлурги ческого холдинга м...,"[совладелец, горно, металлурги, ческого, холди..."
2998,science,Nokia выпустит ОС для бюджетных смартфонов,компания nokia разрабатывает операционную сист...,компания nokia разрабатывает операционную сист...,"[компания, nokia, разрабатывает, операционную,..."


In [405]:
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

Классификация алгоритмом логистической регресии

In [391]:
lr_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr_estimator', LogisticRegression(random_state=8,max_iter=1000))])
lr_clf.fit(train_df.merged_text_tokens, train_df.label)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr_estimator',
                 LogisticRegression(max_iter=1000, random_state=8))])

In [392]:
predicted_lr = lr_clf.predict(test_df.merged_text_tokens)
print(metrics.classification_report(predicted_lr, test_df.label))

              precision    recall  f1-score   support

    business       0.22      0.71      0.34        28
     culture       0.93      0.93      0.93       429
   economics       0.91      0.83      0.87       465
      forces       0.88      0.81      0.85       266
        life       0.91      0.80      0.85       471
       media       0.86      0.83      0.85       416
     science       0.85      0.90      0.87       441
       sport       0.96      0.98      0.97       416
       style       0.75      0.97      0.85        40
      travel       0.48      0.93      0.63        28

    accuracy                           0.87      3000
   macro avg       0.78      0.87      0.80      3000
weighted avg       0.89      0.87      0.88      3000



In [393]:
params_lr = {'lr_estimator__C': [50.0, 20.0, 10.0, 5.0, 1.0, 0.5]}
grid_lr = GridSearchCV(estimator=lr_clf, param_grid=params_lr)
grid_lr.fit(train_df.merged_text_tokens, train_df.label)
print("Лучшее значение параметра: ", grid_lr.best_params_)
print("Лучший результат модели: ", grid_lr.best_score_)

Лучшее значение параметра:  {'lr_estimator__C': 50.0}
Лучший результат модели:  0.8702666666666667


In [394]:
predicted_grid_lr = grid_lr.predict(test_df.merged_text_tokens)
print(metrics.classification_report(predicted_grid_lr, test_df.label))

              precision    recall  f1-score   support

    business       0.47      0.69      0.56        61
     culture       0.92      0.92      0.92       425
   economics       0.92      0.87      0.89       449
      forces       0.88      0.84      0.86       256
        life       0.90      0.83      0.87       449
       media       0.86      0.86      0.86       402
     science       0.87      0.90      0.88       451
       sport       0.97      0.98      0.98       420
       style       0.87      0.92      0.89        49
      travel       0.67      0.95      0.78        38

    accuracy                           0.89      3000
   macro avg       0.83      0.88      0.85      3000
weighted avg       0.89      0.89      0.89      3000



Классификация наивной баесовской моделью

In [396]:
nb_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb_estimator', MultinomialNB())])
nb_clf.fit(train_df.merged_text_tokens, train_df.label)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('nb_estimator', MultinomialNB())])

In [406]:
predicted_nb = nb_clf.predict(test_df.merged_text_tokens)
print(metrics.classification_report(predicted_nb, test_df.label))

              precision    recall  f1-score   support

    business       0.00      0.00      0.00         0
     culture       0.92      0.87      0.89       453
   economics       0.94      0.73      0.83       546
      forces       0.79      0.83      0.81       234
        life       0.88      0.79      0.83       464
       media       0.83      0.80      0.82       418
     science       0.84      0.87      0.85       450
       sport       0.98      0.97      0.97       431
       style       0.08      1.00      0.14         4
      travel       0.00      0.00      0.00         0

    accuracy                           0.83      3000
   macro avg       0.63      0.69      0.61      3000
weighted avg       0.89      0.83      0.86      3000



In [407]:
params_nb = {'nb_estimator__alpha': [0, 0.5, 1.0, 2.0]}
grid_nb = GridSearchCV(nb_clf, params_nb)
grid_nb.fit(train_df.merged_text_tokens, train_df.label)
print(grid_nb.best_params_, grid_nb.best_score_)

{'nb_estimator__alpha': 0.5} 0.8186666666666668


In [408]:
predicted_grid_nb = grid_nb.predict(test_df.merged_text_tokens)
print(metrics.classification_report(predicted_grid_nb, test_df.label))

              precision    recall  f1-score   support

    business       0.02      0.50      0.04         4
     culture       0.92      0.89      0.90       440
   economics       0.93      0.75      0.83       527
      forces       0.85      0.77      0.81       270
        life       0.88      0.79      0.83       466
       media       0.82      0.80      0.81       411
     science       0.82      0.88      0.85       430
       sport       0.97      0.97      0.97       422
       style       0.42      1.00      0.59        22
      travel       0.15      1.00      0.26         8

    accuracy                           0.84      3000
   macro avg       0.68      0.84      0.69      3000
weighted avg       0.88      0.84      0.85      3000



Метод опорных векторов без поиска лучшей модели

In [399]:
from sklearn import svm

In [400]:
sv = svm.SVC()

In [401]:
sv_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb_estimator', svm.SVC())])
sv_clf.fit(train_df.merged_text_tokens, train_df.label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('nb_estimator', SVC())])

In [403]:
predicted_nb = sv_clf.predict(test_df.merged_text_tokens)
print(metrics.classification_report(predicted_nb, test_df.label))

              precision    recall  f1-score   support

    business       0.18      0.70      0.28        23
     culture       0.92      0.94      0.93       418
   economics       0.92      0.82      0.86       476
      forces       0.90      0.82      0.86       267
        life       0.93      0.80      0.86       485
       media       0.86      0.83      0.85       418
     science       0.85      0.91      0.88       431
       sport       0.96      0.98      0.97       414
       style       0.75      0.95      0.84        41
      travel       0.48      0.96      0.64        27

    accuracy                           0.87      3000
   macro avg       0.77      0.87      0.80      3000
weighted avg       0.89      0.87      0.88      3000

